
# <center> **Implementación de una red Bayesiana de desición**
---
<center> por: José Miguel Muñoz Arias


In [103]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

Partimos de la inferencia Bayesiana 
$$P(A|B)=\frac{P(B|A)P(A)}{P(B)}$$

O escrita de tal forma que de cuenta de las distribuciones  de probabilidad:

$$P(\theta|B)=\frac{P(B|\theta)P(\theta)}{P(B)}$$
Donde estamos interesados en encontrar la mejor distribución $\theta$ para explicar los datos.

Para explicar la forma en que los datos se ajustan a los datos voy a usar cadenas de Markov. Tal vez el más famosos es Metropolis-Hastings. 

El algoritmo dice:


---


> 1. Seleccionar un valor inicial $\theta_0$
> 2. Para i>m, calcular:<br>
> Tomar un valor $\theta^* => q(\theta^*|\theta_i)$<br>
> Calcular $\alpha=\frac{g(\theta^*)q(\theta^*|\theta_i)}{g(\theta_{i-1})q(\theta^*|\theta_{i-1})}$<br>
> Si $\alpha \geq 1 \text{  aceptar } \alpha\ \Rightarrow \theta_i \leftarrow \  \theta^*$<br>
> Si $\alpha \in [0,1) \text{  aceptar } \alpha\ \text{ con probabilidad }p \Rightarrow \theta_i \leftarrow \  \theta^*$



---
Una forma alternativa es ir directamente con el uso de la distribución $q $ uniforme, que generalmente mejora el tiempo computacional:


---


> 1. Seleccionar un valor inicial $\theta_0$
> 2. Para i>m, calcular:<br>
> Tomar un valor $\alpha \rightarrow U_{[0,1]}$<br>
> Tomar $\theta^*\rightarrow q(\theta^*|\theta_i)$<br>
> Si $\alpha < min(1, \frac{g(\theta^*)q(\theta^*|\theta_i)}{g(\theta_{i-1})q(\theta^*|\theta_{i-1})}) \Rightarrow \theta_{i+1}=\theta ^*$<br>
> Sinó: $\theta_{i+1}=\theta_{i}$



---


Como siempre, para no complicar el modelo, podemos suponer distribuciones exponenciales con parámetros $\mu_x$ y $\sigma_x$. Pero es MUY importante, antes de implementar el modelo ver la distribución a la que se asemejan los datos antes de intentar ajustar. 


In [104]:
class MetropolisHastings:

  def __init__ (self,valores_Iniciales,datos,Transicion="Normal",iteraciones=1000):
    if Transicion=="Normal":
      self.Transicion=lambda x: [x[0],np.random.normal(x[1],0.5,(1,))]
    elif Transicion=="Uniforme":
      self.Transicion=lambda x: [x[0],np.random.uniform(x[1],0.5)]
    self.param_init=valores_Iniciales
    self.data=datos
    self.iteraciones=iteraciones
  def prior(self,x):
    #x[0] = mu, x[1]=sigma (new or current)
    if x[1] <=0:
        return 0
    else: return 1

  #Calculamos el log Likelihood
  def log_like_normal(self,x,data):
    #x[0]=mu, x[1]=sigma (new or current)
    return np.sum(-np.log(x[1] * np.sqrt(2* np.pi) +1 )-((data-x[0])**2) / (2*x[1]**2))


  #Defines whether to accept or reject the new sample
  def acceptance(self,x, x_new):
    if x_new>x:  return True
    else:
        accept=np.random.uniform(0,1)
        return (accept < (np.exp(x_new-x)))


  def ajustar(self):
    x = self.param_init
    accepted = []
    rejected = []   
    for i in range(self.iteraciones):
        x_new =  self.Transicion(x)    
        x_lik = self.log_like_normal(x,self.data)
        x_new_lik = self.log_like_normal(x_new,self.data) 
        if (self.acceptance(x_lik + np.log(self.prior(x)+1),x_new_lik+np.log(self.prior(x_new)+1))):            
            x = x_new
            accepted.append(x_new)
        else:   rejected.append(x_new)            
    media_accepted=(np.array(accepted).T[0]);des_accepted=[np.array(accepted).T[1,i][0] for i in range(len(accepted))]
    if len(rejected)!=0:   media_rejected=(np.array(rejected).T[0]);des_rejected=[np.array(rejected).T[1,i][0] for i in range(len(rejected))]
    else: media_rejected=[0];des_rejected=[0]
    dfACC=pd.DataFrame({"Media":media_accepted,"Desviación":des_accepted}).dropna()
    dfNEG=pd.DataFrame({"Media":media_rejected,"Desviación":des_rejected}).dropna()
    print("*"*20);print();print("Valores Aceptados por el MonteCarlo: ");print(np.mean(dfACC["Media"]),np.mean(dfACC["Desviación"]))
    return dfACC, dfNEG

## Aplicación al crédito


In [105]:
#Tomo las columnas numéricas
df=pd.read_excel("/content/datosCredito.xlsx")[['Edad', 'Estado civil', 'Hijos', 'Estrato', 'Nivel de estudios', 'Ingresos', 'Egresos', 'Califica', 'Puntaje']]
df[["Puntaje"]]=df[["Puntaje"]].dropna()
df.head()

,Edad,Estado civil,Hijos,Estrato,Nivel de estudios,Ingresos,Egresos,Califica,Puntaje
0,22,Soltero,0,2,Técnico,949000,226000,1,27.0
1,57,Casado,2,4,Bachiller,1806000,1319000,1,63.0
2,21,Casado,0,2,Bachiller,816000,157000,0,NaN
3,23,Soltero,0,2,Técnico,583000,42000,1,17.0
4,22,Soltero,0,2,Técnico,750000,208000,1,20.0


In [106]:
#Para solucionar el problema de las variables categoricas:
#Para el estado civil:
estado_civil={"Soltero":0,"Casado":1,"Unión Libre":2,"Separado":3,"Viudo": 11}
df[["Estado civil"]]=df[["Estado civil"]].replace(estado_civil)
#Para el nivel de estudios
estudios={"Bachiller":2,"Técnico": 3,"Profesional":5,"Tecnólogo":4,"Primaria":1,"Especializacion":6,"Ninguno": 0}
df[["Nivel de estudios"]]=df[["Nivel de estudios"]].replace(estudios)
#Veamos cómo quedó
df.head()

,Edad,Estado civil,Hijos,Estrato,Nivel de estudios,Ingresos,Egresos,Califica,Puntaje
0,22,0,0,2,3,949000,226000,1,27.0
1,57,1,2,4,2,1806000,1319000,1,63.0
2,21,1,0,2,2,816000,157000,0,NaN
3,23,0,0,2,3,583000,42000,1,17.0
4,22,0,0,2,3,750000,208000,1,20.0


Ahora que puedo estimar bien la media de cada evento y su desviación usando el MCCM, puedo empezar a construir una matriz de confusión, como primer ejemplo propongo la edad


In [107]:
#Probabilidad de que califique y la edad sea más grande que la media:
#Calculo de la media de la edad:
datos=np.array(df["Edad"])
mu_obs=np.mean(datos);std_obs=np.std(datos)
print(mu_obs,std_obs)
MCCM=MetropolisHastings([mu_obs,std_obs],datos)
accepted, rejected = MCCM.ajustar()

#Tomando los valores de MCCM:
mu_calc_edad=40.727272727272265;sig_calc_edad=12.509422024657363
#Probabilidad condicional para la edad

df[((np.abs(df["Edad"]-mu_calc_edad)/sig_calc_edad>=0.5))].groupby("Califica").size()


40.72727272727273 12.162052410042055


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


********************

Valores Aceptados por el MonteCarlo: 
40.727272727272265 12.381833004849184


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Califica
0     32
1    104
dtype: int64

Como segundo ejemplo, el estado civil:

In [110]:
#Probabilidad de que califique y la edad sea más grande que la media:
#Calculo de la media del estado civil:
datos=np.array(df["Estado civil"])
mu_obs=np.mean(datos);std_obs=np.std(datos)
MCCM=MetropolisHastings([mu_obs,std_obs],datos)
accepted, rejected = MCCM.ajustar()

#Tomando los valores de MCCM:
mu_calc_est=np.mean(accepted["Media"]);sig_calc_est=np.mean(accepted["Desviación"])
#Probabilidad condicional para la estado civil:
print();print("Calificaciones por encima de la media:")
val_encima=df[((df["Estado civil"]>=mu_calc_est))].groupby("Califica").size()
print(val_encima)
print();print("Calificaciones por debajo de la media:")
val_debajo=df[((df["Estado civil"]<mu_calc_est))].groupby("Califica").size()
print(val_debajo)

print();print("*"*20);print("Con probabilidades: ")

matriz=np.array([list(np.array(val_encima)/len(df["Califica"])),list(np.array(val_debajo)/len(df["Califica"]))])
print(matriz)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


********************

Valores Aceptados por el MonteCarlo: 
1.3939393939393887 2.6882950613609506

Calificaciones por encima de la media:
Califica
0     8
1    36
dtype: int64

Calificaciones por debajo de la media:
Califica
0     40
1    114
dtype: int64

********************
Con probabilidades: 
[[0.04040404 0.18181818]
 [0.2020202  0.57575758]]


### De forma Generalizada

Ingresar la variable con la que quiera hacer el estudio


In [113]:
vari=input("Ingrese la variable ")

#Probabilidad de que califique y la edad sea más grande que la media:
#Calculo de la media del estado civil:
datos=np.array(df[vari])
mu_obs=np.mean(datos);std_obs=np.std(datos)
MCCM=MetropolisHastings([mu_obs,std_obs],datos)
accepted, rejected = MCCM.ajustar()

#Tomando los valores de MCCM:
mu_calc_est=np.mean(accepted["Media"]);sig_calc_est=np.mean(accepted["Desviación"])
#Probabilidad condicional para la estado civil:
condicion= lambda val: np.abs(val-mu_calc_est)/sig_calc_est
print();print("Calificaciones por encima de la media:")
val_encima=df[((condicion(df[vari])>=0.5))].groupby("Califica").size()
print(val_encima)
print();print("Calificaciones por debajo de la media:")
val_debajo=df[((condicion(df[vari])<0.5))].groupby("Califica").size()
print(val_debajo)

print();print("*"*20);print("Con probabilidades, la matriz de confusión {}: ".format(vari))

matriz=np.array([list(np.array(val_encima)/len(df["Califica"])),list(np.array(val_debajo)/len(df["Califica"]))])
print(matriz)


Ingrese la variable Ingresos


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


********************

Valores Aceptados por el MonteCarlo: 
1635497.555555577 1217701.6340647575

Calificaciones por encima de la media:
Califica
0     32
1    102
dtype: int64

Calificaciones por debajo de la media:
Califica
0    16
1    48
dtype: int64

********************
Con probabilidades, la matriz de confusión Ingresos: 
[[0.16161616 0.51515152]
 [0.08080808 0.24242424]]


Gracias!